In [52]:
import matplotlib.pyplot as plt
import numpy as np
import PhysicsFunctions as phy

In [53]:
# Constants
b = 8.2 * 10**(-3) #Pa*m

p =101625.52 #PA assumed not actually measured  #atmospheric pressure
distance = .00763 #m 
distance_unc = 0.005 * 10**(-3)
oil_density = 886 #kg/m^3
air_viscosity = 1.8370 *10**(-5) #Nsm^-2 *10^-5
g = 9.81 #m/s^2

temp = 22.899283 #C

In [54]:

## Measured Data
Voltage = 499 #V
voltage_unc = 0.5 #V
time_unc = 0.005

t_rise = np.array([[4.42,4.24,4.24,4.29,4.2,4.66,4.07,4.22,4.25,4.19,4.57,4.25,4.26,4.28,4.39,4.32,4.53,4.63,4.1],
                  [12.13,14.05,11.69,12.66,12.48,12.58,13.4,13.59,12.92,11.67,11.32],
                  [1.49,1.52,1.72,1.37,1.44,2.01,2.09,1.98,1.88,1.81],
                  [1.09,1.06,1.06,1.01,1.17,1.18,1.23,1.24,1.26,1.19],
                  [4.73,4.08,4.45]],dtype = object)
          
t_fall = np.array([[20,20.35,15.07,22.2,18.25,20.05,16.82,16.56,18.59,16.98,17.56,16.49,16.64,19.47,17.62,16.32,18.22,18.2,16.61],
          [15.13,15.82,17.19,13.98,18.04,14.99,16.53,17.11,17.26,15.31,18.51],
          [17.43,17.58,18.96,16.23,17.77,16.78,15.55,17.02,17.11,16.01],
          [16.72,16.14,18.52,17.85,16.54,18.69,20.34,17.17,17.06,16.3],
          [18.11,16.03,18.04]],dtype = object)
d_reticle = 0.0005 #m


In [80]:
# Drop Velocity Calculations
v_rise = []
for l in t_rise: 
    v_rise.append(d_reticle/np.array(l))

v_fall = []
for l in t_fall: 
    v_fall.append(d_reticle/np.array(l))
    
    
# ------------ UNCERTAINTY
v_rise_unc = []
for i in range(0,len(v_rise),1):
    v_rise_unc.append(v_rise[i] - d_reticle/(np.array(t_rise[i])+time_unc))

v_fall_unc = []
for i in range(0,len(v_fall),1): 
    v_fall_unc.append(v_fall[i] - d_reticle/(np.array(t_rise[i])+time_unc))
    
    

#print(v_rise,v_fall)
    



In [104]:

#Drop Radius Calculation

drop_radius_uncorrected = []

for i in range(0,len(v_fall),1):
    drop_radius_uncorrected.append(np.sqrt(9*air_viscosity*v_fall[i] / (2*oil_density*g)))


drop_radius_uncorrected_unc = []

for i in range(0,len(v_fall),1):
    drop_radius_uncorrected_unc.append(drop_radius_uncorrected[i] - np.sqrt(9*air_viscosity*(v_fall[i]-v_fall_unc[i]) / (2*oil_density*g)))



#get corrected viscosity

viscosity = []
for i in range(0,len(drop_radius_uncorrected),1):
    viscosity.append(air_viscosity*(1/(1+b/(p*drop_radius_uncorrected[i]))))

viscosity_unc = []
for i in range(0,len(drop_radius_uncorrected),1):
    viscosity_unc.append(viscosity[i] - air_viscosity*(1/(1+b/(p*(drop_radius_uncorrected[i]+ drop_radius_uncorrected_unc[i])))))
    
#get new drop radius
drop_radius =[]
for i in range(0,len(v_fall),1):
    drop_radius.append(np.sqrt(9*viscosity[i]*v_fall[i] / (2*oil_density*g)))
    
drop_radius_unc =[]
for i in range(0,len(v_fall),1):
    drop_radius_unc.append(drop_radius[i] - np.sqrt(np.abs(9*(viscosity[i]-viscosity_unc[i])*(v_fall[i]-v_fall_unc[i]) / (2*oil_density*g))))
                   


In [106]:



# Calculations 

E_field = Voltage/distance
E_field_unc = E_field - (Voltage+voltage_unc)/(distance-distance_unc)

#Charge of Droplet
q = []
for i in range(0,len(viscosity),1):
    q.append(4*np.pi*(drop_radius[i]**3) * oil_density * g * (v_fall[i] + v_rise[i]) / (3* E_field *v_fall[i]))

q_unc = []
for i in range(0,len(viscosity),1):
    q_unc.append(q[i] - 4*np.pi*((drop_radius[i]+drop_radius_unc[i])**3) * oil_density * g * (v_fall[i]+v_fall_unc[i] + v_rise[i]+v_fall_unc[i]) / (3* (E_field-E_field_unc) *(v_fall[i]+v_fall_unc[i])))

for i in range(0,len(q),1):
    print("$",q[i].mean(), "\pm", q_unc[i].mean(),"$")
    
#Alternate Calculation
q_alt = []
for i in range(0,len(v_fall),1):
    q_alt.append((4*np.pi/3)*(np.sqrt((b/(2*p))**2+9*air_viscosity*v_fall[i]/(2*oil_density*g))-(b/(2*p)))**3 *(oil_density*g*distance*(v_fall[i] + v_rise[i])/(Voltage*v_fall[i])))

 
q_alt_unc = []
for i in range(0,len(v_fall),1):
    q_alt_unc.append(q[i] - (4*np.pi/3)*(np.sqrt((b/(2*p))**2+9*air_viscosity*(v_fall[i]-v_fall_unc[i])/(2*oil_density*g))-(b/(2*p)))**3 *(oil_density*g*(distance-distance_unc)*(v_fall[i]-v_fall_unc[i] + v_rise[i]-v_rise_unc[i])/(Voltage*v_fall[i]-v_fall_unc[i])))

    
print()
for i in range(0,len(q_alt),1):
    print("$",q_alt[i].mean(), "\pm", np.abs(q_alt_unc[i].mean()),"$")
#print(q)

#print(q_unc)

#print(q_alt)



$ 3.1641755473521804e-19 \pm 5.126874982796171e-19 $
$ 1.6447282387370168e-19 \pm 4.5456952385838774e-20 $
$ 7.332210570931292e-19 \pm 1.015009820304562e-18 $
$ 1.0384513046349205e-18 \pm 1.8564213276553762e-18 $
$ 3.1891539794396863e-19 \pm 2.581948672036107e-19 $

$ 3.112769696616874e-19 \pm 4.4500122731845465e-18 $
$ 1.620281942145976e-19 \pm 1.173642156772115e-19 $
$ 7.218141369442769e-19 \pm 4.991495692349997e-17 $
$ 1.0219244984481665e-18 \pm 1.3430233946193704e-16 $
$ 3.1388384661956774e-19 \pm 4.0535464052798345e-18 $


In [113]:
ratio = []
for arr in q:
    ratio.append(arr.mean()/q[1].mean())
    
ratio_unc = []
for i in range(0,len(q),1):
    ratio_unc.append(ratio[i] - (q[i].mean()- q_unc[i].mean())/(q_unc[1].mean()+q[1].mean()))
    
#ratio.sort()
for i in range(0,len(ratio),1):
    print("$",ratio[i], "\pm", np.abs(ratio_unc[i]),"$")



$ 1.9238287960459315 \pm 2.858760214739707 $
$ 1.0 \pm 0.43306817351759963 $
$ 4.458007346284563 \pm 5.800307463172468 $
$ 6.313816958796457 \pm 10.210215257288343 $
$ 1.939015762195845 \pm 1.6497736555149523 $


# Accepted Value
1.60 * 10^-19